# LoRA Edge Vision - End-to-End Demo

This script demonstrates the full workflow of LoRA fine-tuning of Stable Diffusion for 
aerial imagery, optimized for edge deployment with ONNX.

## Project Overview

This project showcases:
- Low-Rank Adaptation (LoRA) fine-tuning of Stable Diffusion for aerial/satellite imagery
-  Memory-efficient training techniques
- Exporting models to ONNX format for edge deployment
- Performance comparison between PyTorch and ONNX inference

In [ ]:
import os
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import onnxruntime as ort
from diffusers import StableDiffusionPipeline
from peft import PeftModel

# Set paths to our models
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
LORA_PATH = "models/lora_adapters"
MERGED_MODEL_PATH = "models/sd_lora_pipeline"
ONNX_DIR = "onnx_mac"
OUTPUT_DIR = "demo_outputs"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Detect device
if torch.cuda.is_available():
    DEVICE = "cuda"
elif torch.backends.mps.is_available():
    DEVICE = "mps"
else:
    DEVICE = "cpu"

print(f"Using device: {DEVICE}")

### 1. Project Architecture

The overall architecture of the project follows these steps:
1. Precompute VAE latents from aerial imagery
2. Train LoRA adapter on latents
3. Merge LoRA adapter with base model
4. Export to ONNX for edge deployment
5. Run inference on edge devices

### 2. Load and Test PyTorch Model with LoRA

First, we load our LoRA-adapted model in PyTorch and generate sample images.


In [ ]:
def load_pytorch_model(base_model, lora_adapter, device="cpu"):
  """Load PyTorch model with LoRA adapter"""
  pipe = StableDiffusionPipeline.from_pretrained(
    base_model,
    torch_dtype=torch.float32
  ).to(device)

  pipe.enable_attention_slicing()
  pipe.unet = PeftModel.from_pretrained(pipe.unet, lora_adapter)

  return pipe

def generate_pytorch(pipe, prompt, steps=30, scale=7.5, seed=None):
  """Generate image using PyTorch model"""
  generator = None
  if seed is not None:
    generator = torch.Generator(device=pipe.device).manual_seed(seed)

  start_time = time.time()
  image = pipe(
    prompt,
    num_inference_steps=steps,
    guidance_scale=scale,
    generator=generator
  ).images[0]
  end_time = time.time()

  return image, end_time - start_time

# Load the PyTorch model with LoRA adapter
print("Loading PyTorch model with LoRA adapter...")
pytorch_pipe = load_pytorch_model(BASE_MODEL, LORA_PATH, DEVICE)
print("Model loaded successfully!")

# Test prompt ideas
test_prompts = [
  "Aerial view of agricultural fields with irrigation patterns",
  "Satellite imagery of coastal urban area with beaches",
  "Bird's eye view of mountain landscape with snow caps",
  "Top-down view of desert patterns with sand dunes",
  "Aerial photograph of river delta with sediment patterns"
]

# Generate sample image with PyTorch model
print("Generating sample image with PyTorch model...")
seed = 42  # For reproducibility
sample_prompt = test_prompts[0]
pytorch_image, pytorch_time = generate_pytorch(pytorch_pipe, sample_prompt, steps=30, seed=seed)

# Save the image
pytorch_path = os.path.join(OUTPUT_DIR, "pytorch_sample.png")
pytorch_image.save(pytorch_path)
print(f"PyTorch generation took {pytorch_time:.2f} seconds")
print(f"Sample image saved to {pytorch_path}")

### 3. Load and Test ONNX Model

Now, we test the ONNX version of our model, which is optimized for edge deployment.

In [ ]:
def load_onnx_components(onnx_dir, base_model):
  """Load ONNX UNet and other components"""
  from diffusers import DDIMScheduler

  # Set up ONNX Runtime session
  sess_options = ort.SessionOptions()
  sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL

  # Load UNet ONNX model
  unet_path = os.path.join(onnx_dir, "unet.onnx")
  unet_session = ort.InferenceSession(unet_path, sess_options)

  # Load other components from PyTorch
  sd_pipe = StableDiffusionPipeline.from_pretrained(base_model, torch_dtype=torch.float32)

  return {
    "unet": unet_session,
    "vae": sd_pipe.vae,
    "text_encoder": sd_pipe.text_encoder,
    "tokenizer": sd_pipe.tokenizer,
    "scheduler": DDIMScheduler.from_pretrained(base_model, subfolder="scheduler")
  }

In [ ]:
def generate_onnx(components, prompt, steps=30, scale=7.5, seed=None):
  """Generate image using ONNX model"""
  import torch.nn.functional as F

  # Set random seed for reproducibility
  if seed is not None:
    np.random.seed(seed)
    torch.manual_seed(seed)

  unet = components["unet"]
  vae = components["vae"]
  text_encoder = components["text_encoder"]
  tokenizer = components["tokenizer"]
  scheduler = components["scheduler"]

  # Get UNet input and output names
  unet_inputs = unet.get_inputs()
  unet_output = unet.get_outputs()[0].name

  # Process text input
  text_input = tokenizer(
    [prompt],
    padding="max_length",
    max_length=tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt"
  )

  with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids)[0].numpy()

  # Unconditional embeddings for guidance
  uncond_input = tokenizer(
    [""],
    padding="max_length",
    max_length=tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt"
  )

  with torch.no_grad():
    uncond_embeddings = text_encoder(uncond_input.input_ids)[0].numpy()

  # Concatenate for classifier-free guidance
  text_embeddings = np.concatenate([uncond_embeddings, text_embeddings])

  # Setup scheduler
  scheduler.set_timesteps(steps)

  # Create random latents
  latents_shape = (1, 4, 64, 64)  # Standard for 512x512 images
  latents = np.random.randn(*latents_shape).astype(np.float32)
  latents = latents * scheduler.init_noise_sigma

  start_time = time.time()

  # Denoising loop
  for i, t in enumerate(scheduler.timesteps):
    # Duplicate latents for classifier-free guidance
    latent_model_input = np.repeat(latents, 2, axis=0)

    # Get timestep
    timestep = np.array([t]).astype(np.int64)

    # Get input names based on exported model
    onnx_inputs = {}
    for input_info in unet_inputs:
        if "sample" in input_info.name.lower():
            onnx_inputs[input_info.name] = latent_model_input
        elif "timestep" in input_info.name.lower():
            onnx_inputs[input_info.name] = timestep
        elif "encoder_hidden_states" in input_info.name.lower():
            onnx_inputs[input_info.name] = text_embeddings

    # Run inference
    noise_pred = unet.run([unet_output], onnx_inputs)[0]

    # Perform guidance
    noise_pred_uncond, noise_pred_text = np.split(noise_pred, 2)
    noise_pred = noise_pred_uncond + scale * (noise_pred_text - noise_pred_uncond)

    # Compute previous noisy sample
    latents = scheduler.step(
        torch.from_numpy(noise_pred),
        t,
        torch.from_numpy(latents)
    ).prev_sample.numpy()

  # Decode latents
  latents = 1 / 0.18215 * latents

  with torch.no_grad():
    latents_torch = torch.from_numpy(latents)
    image = vae.decode(latents_torch).sample

  # Convert to image
  image = (image / 2 + 0.5).clamp(0, 1)
  image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
  image = (image * 255).round().astype("uint8")[0]

  end_time = time.time()

  return Image.fromarray(image), end_time - start_time

In [ ]:
# Load ONNX components
print("Loading ONNX model components...")
onnx_components = load_onnx_components(ONNX_DIR, BASE_MODEL)
print("ONNX components loaded successfully!")

# Generate sample image with ONNX model
print("Generating sample image with ONNX model...")
onnx_image, onnx_time = generate_onnx(onnx_components, sample_prompt, steps=30, seed=seed)

# Save and display the image
onnx_path = os.path.join(OUTPUT_DIR, "onnx_sample.png")
onnx_image.save(onnx_path)
print(f"ONNX generation took {onnx_time:.2f} seconds")
print(f"ONNX sample saved to {onnx_path}")


### 4. Performance Comparison

Let's compare the inference speed of PyTorch vs. ONNX models.

In [ ]:
def run_benchmark(models, prompts, steps=30, trials=3):
  """Benchmark PyTorch vs ONNX performance"""
  results = {"pytorch": [], "onnx": []}

  for prompt in prompts:
    print(f"Benchmarking prompt: {prompt}")

    # PyTorch benchmark
    pytorch_times = []
    for i in range(trials):
      _, time_taken = generate_pytorch(models["pytorch"], prompt, steps=steps, seed=None)
      pytorch_times.append(time_taken)
      print(f"  PyTorch trial {i+1}: {time_taken:.2f}s")

    # ONNX benchmark
    onnx_times = []
    for i in range(trials):
      _, time_taken = generate_onnx(models["onnx"], prompt, steps=steps, seed=None)
      onnx_times.append(time_taken)
      print(f"  ONNX trial {i+1}: {time_taken:.2f}s")

    # Record average times
    results["pytorch"].append(sum(pytorch_times) / len(pytorch_times))
    results["onnx"].append(sum(onnx_times) / len(onnx_times))

  return results, prompts

# For a quick benchmark, let's select two prompts
benchmark_prompts = test_prompts[:2]

# Create models dictionary
models = {
  "pytorch": pytorch_pipe,
  "onnx": onnx_components
}

# Run benchmark
print("Running performance benchmark (this may take a while)...")
benchmark_results, benchmark_prompts = run_benchmark(models, benchmark_prompts, steps=20, trials=2)

# Visualize benchmark results
plt.figure(figsize=(10, 6))

x = np.arange(len(benchmark_prompts))
width = 0.35

pytorch_times = benchmark_results["pytorch"]
onnx_times = benchmark_results["onnx"]

plt.bar(x - width/2, pytorch_times, width, label='PyTorch')
plt.bar(x + width/2, onnx_times, width, label='ONNX')

plt.ylabel('Inference Time (seconds)')
plt.title('PyTorch vs ONNX Inference Performance')
plt.xticks(x, [f"Prompt {i+1}" for i in range(len(benchmark_prompts))])
plt.legend()

In [ ]:
# Add speedup percentage labels
for i, (pt, ox) in enumerate(zip(pytorch_times, onnx_times)):
  speedup = (pt - ox) / pt * 100
  plt.annotate(f"{speedup:.1f}% faster",
              xy=(i + width/2, ox),
              xytext=(0, 10),
              textcoords="offset points",
              ha='center')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "benchmark.png"))
plt.show()

### 5. Visual Comparison

Let's compare the visual quality of PyTorch vs. ONNX generations to ensure the edge-optimized model maintains quality.

In [ ]:
def compare_generations(pytorch_pipe, onnx_components, prompts, steps=30, seed=42):
  """Generate and compare images from PyTorch and ONNX models"""
  plt.figure(figsize=(15, 5 * len(prompts)))

  for i, prompt in enumerate(prompts):
    print(f"Generating comparison for: {prompt}")

    # Generate with PyTorch
    torch_img, torch_time = generate_pytorch(pytorch_pipe, prompt, steps=steps, seed=seed)

    # Generate with ONNX
    onnx_img, onnx_time = generate_onnx(onnx_components, prompt, steps=steps, seed=seed)

    # Plot results
    plt.subplot(len(prompts), 2, i*2 + 1)
    plt.imshow(np.array(torch_img))
    plt.title(f"PyTorch: {torch_time:.2f}s")
    plt.axis('off')

    plt.subplot(len(prompts), 2, i*2 + 2)
    plt.imshow(np.array(onnx_img))
    plt.title(f"ONNX: {onnx_time:.2f}s")
    plt.axis('off')

    # Save individual images
    torch_img.save(os.path.join(OUTPUT_DIR, f"pytorch_sample_{i}.png"))
    onnx_img.save(os.path.join(OUTPUT_DIR, f"onnx_sample_{i}.png"))

  plt.tight_layout()
  plt.savefig(os.path.join(OUTPUT_DIR, "visual_comparison.png"))
  plt.show()

# Compare generations for selected prompts
compare_prompts = test_prompts[2:4]  # Use a couple different prompts
compare_generations(pytorch_pipe, onnx_components, compare_prompts)

### 6. Memory Usage
Let's also compare the memory usage of PyTorch vs. ONNX models, which is critical for edge deployment.

In [ ]:
def get_model_size(model_path):
  """Get size of model files in MB"""
  if os.path.isfile(model_path):
    return os.path.getsize(model_path) / (1024 * 1024)

  total_size = 0
  for dirpath, dirnames, filenames in os.walk(model_path):
    for f in filenames:
      fp = os.path.join(dirpath, f)
      total_size += os.path.getsize(fp)

  return total_size / (1024 * 1024)

# Get model sizes
pytorch_size = get_model_size(MERGED_MODEL_PATH)
onnx_size = get_model_size(ONNX_DIR)

print(f"PyTorch model size: {pytorch_size:.2f} MB")
print(f"ONNX model size: {onnx_size:.2f} MB")
print(f"Size reduction: {(pytorch_size - onnx_size) / pytorch_size * 100:.2f}%")

# Visualize model sizes
plt.figure(figsize=(8, 5))
plt.bar(["PyTorch", "ONNX"], [pytorch_size, onnx_size])
plt.ylabel("Model Size (MB)")
plt.title("Model Size Comparison")
plt.savefig(os.path.join(OUTPUT_DIR, "size_comparison.png"))
plt.show()

### 7. Gallery of Generated Images

Let's create a gallery of images generated with our LoRA-tuned model to showcase its capabilities for aerial imagery. This demonstrates how our specialization for aerial/satellite photography has enhanced the model's ability to generate this specific type of content.

In [ ]:
def generate_gallery(model, prompts, steps=30):
  """Generate a gallery of images"""
  gallery = []

  for i, prompt in enumerate(prompts):
    print(f"Generating image {i+1}/{len(prompts)}: {prompt}")

    # Set different seed for each image for variety
    seed = 1000 + i

    # Generate with ONNX for speed
    img, _ = generate_onnx(model, prompt, steps=steps, seed=seed)

    # Save the image
    img_path = os.path.join(OUTPUT_DIR, f"gallery_{i}.png")
    img.save(img_path)

    gallery.append((img, prompt))

  return gallery

# Generate gallery
gallery_prompts = [
  "Aerial view of mountain ranges with lakes",
  "Satellite imagery of tropical island with coral reefs",
  "Bird's eye view of urban park with winding paths",
  "Top-down view of agricultural terraces in mountains",
  "Aerial photograph of meandering river through forest"
]

gallery = generate_gallery(onnx_components, gallery_prompts)

# Display gallery
rows = len(gallery) // 2 + len(gallery) % 2
plt.figure(figsize=(15, 5 * rows))

for i, (img, prompt) in enumerate(gallery):
  plt.subplot(rows, 2, i+1)
  plt.imshow(np.array(img))
  plt.title(f"Prompt: {prompt}", fontsize=10)
  plt.axis('off')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "gallery.png"))
plt.show()

### 8. Conclusion and Key Findings

#### Our project demonstrates the successful implementation of:

1. **Efficient Fine-tuning**: Using LoRA to adapt Stable Diffusion for aerial imagery with minimal training resources
2. **Edge Optimization**: Converting the model to ONNX format for efficient deployment on edge devices
3. **Performance Improvements**: 
  - Faster inference time with ONNX Runtime
  - Reduced memory footprint
  - Maintained visual quality

This approach is particularly valuable for specialized aerial imagery applications that need to run efficiently on edge devices with limited computing resources.